<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/Pandas_Analyze_Maintenance_Log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!wget https://raw.githubusercontent.com/profteachkids/CHE5136_Fall2021/main/maintenance_log.txt

--2021-08-28 16:39:33--  https://raw.githubusercontent.com/profteachkids/CHE5136_Fall2021/main/maintenance_log.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26025250 (25M) [text/plain]
Saving to: ‘maintenance_log.txt’

maintenance_log.txt 100%[===================>]  24.82M   132MB/s    in 0.2s    

2021-08-28 16:39:35 (132 MB/s) - ‘maintenance_log.txt’ saved [26025250/26025250]



In [55]:
import pandas as pd

In [56]:
df=pd.read_csv('/content/maintenance_log.txt')
df.head()

,start,end,equipment,task,name,cost
0,2018-10-26,2018-11-09,ReactorA,Overhaul,Corrine,791158.32
1,2018-02-19,2018-03-03,TankE,Inspect,Corrine,88.50
2,2020-04-05,2020-04-09,HeaterC,Inspect,Garek,42.78
3,2019-07-15,2019-07-24,ReactorD,Overhaul,George,570538.03
4,2020-07-29,2020-08-02,ReactorE,Overhaul,Corrine,925844.80


In [57]:
df=df.astype({'equipment':'category','name':'category','cost':'float64','start':'datetime64[ns]', 'end':'datetime64[ns]'})
df.dtypes


start        datetime64[ns]
end          datetime64[ns]
equipment          category
task                 object
name               category
cost                float64
dtype: object

In [58]:
df['days']=(df['end']-df['start']).dt.days

In [59]:
df2=df.set_index(['start','equipment','task','name'])
df2.sort_index(inplace=True)
idx = pd.IndexSlice

In [ ]:
df2.loc[idx['2020-05-30':'2020-06-5',['ColumnA','TankA'],'Inspect',:]]

In [64]:
df3=pd.pivot_table(data=df,values=['cost','days'],columns=['equipment','task','name'],aggfunc=['mean','min','max'])

In [65]:
df3

mean                ...       max          
equipment       ColumnA                ...     TankE          
task              Clean                ...  Overhaul          
name             Carlos       Corrine  ...     Garek    George
cost       60144.905952  32009.036082  ...  14033.56  18094.94
days          15.264848     14.986998  ...     29.00     29.00

[2 rows x 900 columns]

In [81]:
df4=df3.stack([1,2,3])

In [90]:
df5=df4[['mean','max','min']]
df5['mean']=df5['mean'].round(2)

In [91]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(df5.loc[idx['cost']])

                                  mean         max         min
equipment task     name                                       
ColumnA   Clean    Carlos     60144.91    79446.17    39690.27
                   Corrine    32009.04    42106.09    18588.96
                   Garek      36155.86    48773.48    24556.11
                   George     44057.15    58251.52    30040.48
          Inspect  Carlos      6020.87     7905.60     3827.61
                   Corrine     3196.57     4459.63     2150.85
                   Garek       3601.29     4830.86     2416.30
                   George      4386.65     5624.07     2814.01
          Overhaul Carlos    603459.15   794259.92   396776.99
                   Corrine   320695.46   452068.59   204947.36
                   Garek     360755.72   470618.96   212992.56
                   George    438865.51   630600.23   297077.91
ColumnB   Clean    Carlos     90017.18   116899.34    62486.25
                   Corrine    48059.47    62564.81    3

In [ ]:
df6=df5.unstack(0)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
  print(df6)

                                 mean      min       max
name     task                                           
Cole     Inspect  cost  461922.153846  27868.0  966616.0
         Maintain cost  595759.583333  33997.0  969472.0
         Overhaul cost  458339.171429  19028.0  952616.0
Davis    Inspect  cost  426771.161290  17645.0  972715.0
         Maintain cost  447582.441176  24711.0  979856.0
         Overhaul cost  574343.958333  44487.0  998821.0
Delgado  Inspect  cost  496493.424242  43297.0  996792.0
         Maintain cost  504924.472222  24861.0  992471.0
         Overhaul cost  571080.875000  62705.0  991943.0
Holt     Inspect  cost  543383.828571  40997.0  985170.0
         Maintain cost  477176.785714  25759.0  985868.0
         Overhaul cost  521462.259259  34681.0  920464.0
Johnson  Inspect  cost  395370.480000  36200.0  996912.0
         Maintain cost  638359.130435  42770.0  986280.0
         Overhaul cost  470595.057143  26004.0  936579.0
Pena     Inspect  cost  425252.

In [ ]:
df7=df6.reset_index()

In [ ]:
df7.melt(id_vars=[ 'name','task'], value_vars=['max','min','mean'])

,name,task,variable,value
0,Cole,Inspect,max,966616.000000
1,Cole,Maintain,max,969472.000000
2,Cole,Overhaul,max,952616.000000
3,Davis,Inspect,max,972715.000000
4,Davis,Maintain,max,979856.000000
...,...,...,...,...
85,Stuart,Maintain,mean,556341.961538
86,Stuart,Overhaul,mean,447682.103448
87,Young,Inspect,mean,485589.285714
88,Young,Maintain,mean,468007.476190
